In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from azure.storage.blob import BlobServiceClient
import os

# Configurações de caminhos para camada Silver e Gold
silver_path = f"{os.environ['AZURE_STORAGE_ACCOUNT_URL']}/gold"
gold_path = f"{os.environ['AZURE_STORAGE_ACCOUNT_URL']}/gold"

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Transformar Silver para Gold") \
    .getOrCreate()

# Listar todos os arquivos na camada Silver (com base em arquivos .parquet)
blob_service_client = BlobServiceClient(account_url=os.environ['AZURE_STORAGE_ACCOUNT_URL'])
container_client = blob_service_client.get_container_client("gold")
silver_tables = [blob.name for blob in container_client.list_blobs() if blob.name.endswith(".parquet")]

# Loop para processar cada tabela da camada Silver
for table_path in silver_tables:
    # Nome da tabela com base no caminho do arquivo
    table_name = table_path.split("/")[-1].replace(".parquet", "")

    # Leitura dos dados da camada Silver
    df_silver = spark.read.parquet(f"abfss://gold@{os.environ['AZURE_STORAGE_ACCOUNT_NAME']}.dfs.core.windows.net/{table_path}")

    #TODO: Desenvolver camada Gold
    df_gold = df_silver \

    # Caminho da tabela transformada na camada Gold
    gold_table_path = f"{gold_path}/{table_name}.parquet"

    # Gravação no Azure Data Lake em formato Parquet
    df_gold.write \
        .mode("overwrite") \
        .parquet(f"abfss://gold@{os.environ['AZURE_STORAGE_ACCOUNT_NAME']}.dfs.core.windows.net/{gold_table_path}")

    print(f"Tabela {table_name} transformada e salva em {gold_table_path}")

# Encerrar sessão Spark
spark.stop()
